# Starbucks Capstone Challenge

# Pytorch model

In [1]:
import pandas as pd
import boto3
import sagemaker

### Upload the data to S3

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()

In [3]:
data_dir = 'ML_data'
prefix = 'sagemaker/ML'

role = sagemaker.get_execution_role()
input_data = sagemaker_session.upload_data(path = data_dir, bucket = bucket, key_prefix = prefix)

In [4]:
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'


sagemaker/ML-tuning-HL/output/xgboost-200606-2327-001-4bc8c79b/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-002-3dda3937/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-003-b53854c2/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-004-aa39f51b/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-005-24c675e0/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-006-de089ddf/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-007-edf0fee2/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-008-aa577a27/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-009-d79f5982/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-010-db644b5d/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-011-8f94efef/output/model.tar.gz
sagemaker/ML-tuning-HL/output/xgboost-200606-2327-012-7d6c52f3/output/model.tar.gz
sage

### Build and train the pytorch model

In [5]:
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(

In [16]:
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)
estimator = PyTorch(entry_point='train.py',
                   source_dir='source_pytorch',
                   role=role,
                   framework_version='1.0',
                   train_instance_count=1,
                   train_instance_type='ml.p2.xlarge',
                   output_path=output_path,
                   sagemaker_session=sagemaker_session,
                   hyperparameters={
                       'input_features':18,
                       'hidden_dim':300,
                       'output_dim':1,
                       'epochs':30,
                       'batch-size':1000,
                   })

In [17]:
%%time

s3_input_train = sagemaker.s3_input(s3_data='s3://sagemaker-ap-northeast-1-105243015009/sagemaker/ML/train.csv', content_type='csv')
estimator.fit({'train':s3_input_train})

2020-06-14 07:38:18 Starting - Starting the training job...
2020-06-14 07:38:20 Starting - Launching requested ML instances.........
2020-06-14 07:40:01 Starting - Preparing the instances for training......
2020-06-14 07:41:11 Downloading - Downloading input data...
2020-06-14 07:41:34 Training - Downloading the training image.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-14 07:42:30,829 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-06-14 07:42:30,856 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-14 07:42:30,857 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-14 07:42:31,106 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-06-14 07:42:31,106 sagemaker-containers INFO     Generating setup.cfg
2020-06-14 07:42:31,107 sagemaker-contai

### Deploy the model

In [18]:
%%time

from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(model_data=estimator.model_data,
                    role=role,
                    framework_version='1.0',
                    entry_point='predict.py',
                    source_dir='source_pytorch')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------!CPU times: user 471 ms, sys: 24 ms, total: 495 ms
Wall time: 6min 33s


### Import the test data

In [19]:
import os

test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

### Test the model

In [20]:
test_y_preds = predictor.predict(test_x.to_numpy())

In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y_preds, test_y.values)
from sklearn.metrics import f1_score
f1 = f1_score(test_y.values, test_y_preds)
from sklearn.metrics import precision_score
precision = precision_score(test_y.values, test_y_preds, average='binary')
from sklearn.metrics import recall_score
recall = recall_score(test_y.values, test_y_preds, average='binary')
print('\nAccuracy Score:', accuracy)
print('\nF1 Score:', f1)
print('\nPrecision:', precision)
print('\nRecall:', recall)


Accuracy Score: 0.7126669668317575

F1 Score: 0.7791683488090432

Precision: 0.7444346484461097

Recall: 0.8173018753781004


In [14]:
predictor.delete_endpoint()

In [16]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '5B13F44B62D4181A',
   'HostId': 'sUVo7B7MNB+5N9bqkBjbQPjmexq+LmRTeF88vz9NgJZ0BG7K3qUXxCoKhGiqUjMBe6t3RN7rj3I=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'sUVo7B7MNB+5N9bqkBjbQPjmexq+LmRTeF88vz9NgJZ0BG7K3qUXxCoKhGiqUjMBe6t3RN7rj3I=',
    'x-amz-request-id': '5B13F44B62D4181A',
    'date': 'Sat, 06 Jun 2020 12:19:09 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/ML/sagemaker-pytorch-2020-06-06-11-57-04-927/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker/ML/test_x.csv.out'},
   {'Key': 'sagemaker/ML/train.csv'},
   {'Key': 'sagemaker-pytorch-2020-06-06-11-48-31-756/sourcedir.tar.gz'},
   {'Key': 'sagemaker/ML/std.csv'},
   {'Key': 'sagemaker-pytorch-2020-06-06-11-52-46-361/sourcedir.tar.gz'},
   {'Key': 'sagemaker/ML/test.csv'},
   {'Key': 'sagemaker/ML/sagemaker-pytorch-2020-06-06-